# 1. Data Collection

In questa fase mi sono focalizzato sulla raccolta dei dati e sulla generazione di un grafo che mette in relazione un campione di 28.870 artisti su **last.fm** con i relativi coefficienti di similarità.

Il coefficiente di similarità tra due artisti è relativo, appunto, agli insiemi degli artisti simili relativi a quell'artista. Ne sono stati individuati di due tipi:

- Il **Sample Ratio**, ovvero il rapporto tra la grandezza dell'intersezione degli artisti simili dei due artisti in esame e il numero di artisti campione fissato (quindi ad esempio 120 artisti in comune su 250);

- Il **Jaccard Ratio**, il rapporto tra la medesima intersezione e la grandezza dell'insieme unione (ad esempio, 120 artisti in comune sulla somma di quelli comuni e non).

Lo scopo è quello di generare un file CSV con le informazioni principali dei nodi e degli archi. Il tutto è stato necessario poiché non è stato trovato un file CSV di partenza e quindi è sorta la necessità di generarne uno da zero.

Per far ciò ho sfruttato la potenza della programmazione a oggetti e l'API di last.fm.

Il codice degli oggetti è disponibile nel package ```objects``` in questa cartella. Nota che per questioni di spazio la cache utilizzata per l'elaborazione non è stata inclusa nel repository. Per generare il dataset si avvia il file ```generate.py``` (se si vuole testarlo è sconsigliato date le tempistiche).

## 1.1. Inizializzazione del dataset

Si recuperano i dati presenti nel dataset generato, si calcolano quindi:
* il numero totale di artisti;
* il numero di archi trovati;
* il numero di artisti unici in lista che sono collegati almeno con un altro;
* la percentuale di artisti connessi sul totale.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import csv
import pandas as pd

LINKS_PATH = "data/links.csv"
ARTISTS_PATH = "data/artists.csv"

adf = pd.read_csv(ARTISTS_PATH, encoding="utf-8")
ldf = pd.read_csv(LINKS_PATH, encoding="utf-8")

# Faccio una join delle colonne in modo da contare tutti gli ID presenti in entrambe le colonne IDArtista.
ars = pd.concat([ldf["IDArtista1"], ldf["IDArtista2"]]).unique()
cArs = round((ars.shape[0] / len(adf)) * 100, 2)

print(f"Trovati in totale {len(adf)} artisti.\n{ars.shape[0]} artisti unici, {len(ldf)} archi nel dataset.\nIl {cArs}% degli artisti è connesso con almeno un altro artista.")

## 1.2. Visualizzazione delle tabelle

### Tabella degli artisti

In [ ]:
adf

### Tabella degli archi con i rispettivi ratio

Si è scelto di adottare un Ratio (indipendentemente se Sample o Jaccard) che sia al più superiore a ```50.0```.

La tabella non è ordinata per ```IDArtista``` ma per istante in cui quell'arco è stato trovato.


In [ ]:
ldf

## 1.3. Generazione del grafo

Per fare ciò si opta per la libreria ```networkx```.
Si svolgono i medesimi calcoli come da sopra, ma utilizzando la libreria:

In [ ]:
g = nx.Graph()
g.add_nodes_from(ars)

nodes = g.number_of_nodes()
jaccard = False

with open(LINKS_PATH, "r", encoding="utf-8") as f:
    
    reader = csv.reader(f)
    next(reader)
    
    for parts in reader:
        if len(parts) < 6:
            continue
        
        id1, ar1, id2, ar2, sRatio, jRatio = parts[:6]
        
        try:
            id1 = int(id1)
            id2 = int(id2)
            sRatio = float(sRatio)
            jRatio = float(jRatio)
        except ValueError:
            continue
        
        g.add_edge(id1, id2, sRatio=sRatio, jRatio=jRatio)
        
        g.nodes[id1]["name"] = ar1
        g.nodes[id2]["name"] = ar2

Qui di seguito le generalità del grafo:

In [ ]:
edges = g.number_of_edges()
directed = "" if g.is_directed() else " non"

print(f"{nodes} artisti connessi su {len(adf)} ({round((nodes / len(adf)) * 100, 2)}%).\n{edges} archi nel dataset.\nIl grafo{directed} è orientato.")

Gli artisti connessi sono dunque i seguenti:

In [ ]:
conIDs = set(g.nodes)

conAdfIDs = adf[adf["ID"].isin(conIDs)].sort_values(by='ID', ascending=True)
conAdfIDs

➡️ 2. [Network Analysis](../network_analysis/2_analysis.ipynb)